In [2]:
import numpy as np
import pandas as pd
from const import embedding_path
embeddings = np.load(embedding_path, allow_pickle=True)
df = pd.DataFrame(embeddings, columns=['employee', 'embedding'])
print(df.shape)
print(df)

(20, 2)
    employee                                          embedding
0        huy  [0.22873210906982422, -0.9564796686172485, -2....
1        huy  [-0.1191978007555008, -1.648725986480713, -2.4...
2        huy  [0.5203070640563965, -0.9194456338882446, -2.1...
3        huy  [0.20127424597740173, -0.8754792213439941, -2....
4        huy  [-0.11226420104503632, -0.8447368144989014, -2...
5    dung ai  [0.9970278739929199, -1.1126073598861694, -3.1...
6    dung ai  [0.8800203800201416, -1.1757560968399048, -3.0...
7    dung ai  [1.2486956119537354, -1.0430350303649902, -3.2...
8    dung ai  [1.0159077644348145, -1.186078429222107, -2.83...
9    dung ai  [0.7063747644424438, -1.053912878036499, -2.80...
10  dung iot  [-0.014605514705181122, -0.7461525201797485, -...
11  dung iot  [0.022123202681541443, -0.9281516075134277, -0...
12  dung iot  [-0.35685089230537415, -0.5286805629730225, -1...
13  dung iot  [-0.33963507413864136, -0.3695027530193329, -0...
14  dung iot  [-0.53595376014709

In [ ]:
# check file in snapshots exist
import os
from pathlib import Path
from const import snap_path

def get_img():
   list_path = []
   for root, dirs, files in os.walk(snap_path, topdown=False):
      for name in files:
         path = "./" + os.path.join(root, name)
         list_path.append(path)
   return list_path
   
print(get_img())

In [ ]:
from const import img_path
import os

def remove_file(file_path):
    for root, dirs, files in os.walk(file_path, topdown=False):
        for img in files:
            if (".jpg" in img):
                print(os.path.join(root, img))
                os.remove(os.path.join(root, img))
remove_file(img_path)

In [ ]:
from const import configration_path
def check_configration():
    data = open(configration_path, "r").read()
    if (data.find("/config/tmp/camera") != -1):
        print("File {} is ready to use.".format(configration_path[configration_path.rfind('/')+1:]))
    else:
        print("File {} is missing '/config/tmp/camera' in 'whitelist_external_dirs'".format(configration_path[configration_path.rfind('/')+1:]))
check_configration()

In [ ]:
import cv2
from const import input_shape_size, w_min
from utils import load_database
from custom_deepface.deepface.commons import functions

df, face_cascade = load_database()
list_img_path = '/home/iwin/Desktop/df_non_tf/camera.jpg'
img = cv2.imread(list_img_path)
while(img.shape[0] > input_shape_size):
    img = cv2.resize(img, (int(img.shape[1]/2), int(img.shape[0]/2)))
faces = face_cascade.detectMultiScale(img,  1.3, 5)
for (x, y, w, h) in faces:
    if w > w_min:  # discard small detected faces
        # -------------------------------
        # apply deep learning for custom_face
        base_img = img.copy()
        img, region = functions.detect_face(
            img=img, enforce_detection=False)
        # --------------------------

        if img.shape[0] > 0 and img.shape[1] > 0:
            img = functions.align_face(img=img)
        else:
            img = base_img.copy()
        cv2.imwrite(list_img_path, img)

In [ ]:
from const import input_shape
import numpy as np
from view.utils.lite_predict import predict_tfmodel
from custom_deepface.deepface.commons import distance as dst

# post-processing
img = cv2.resize(img, input_shape)
img_pixels = np.array(img, dtype=np.float32)
face_pixels = np.expand_dims(img_pixels, axis=0)
face_pixels /= 255  # normalize input in [0, 1]

# check preprocess_face function handled
if face_pixels.shape[1:3] == input_shape:
    if df.shape[0] > 0:
        img1_representation = predict_tfmodel(face_pixels)[
            0, :]

        def findDistance(row):
            img2_representation = row['embedding']
            distance = dst.findCosineDistance(
                img1_representation, img2_representation)
            return distance
        df['distance'] = df.apply(findDistance, axis=1)
        df = df.sort_values(by=["distance"])
        list_candidates = df.iloc[0:3]['employee'].tolist()
        if list_candidates.count(list_candidates[0]) >= 2:
            candidate_label = list_candidates[0]
        elif list_candidates.count(list_candidates[1]) >= 2:
            candidate_label = list_candidates[1]
        else:
            candidate_label = 'unknown'
print(candidate_label)